In [1]:
import openai
from qdrant_client import QdrantClient

### Embedding function

In [2]:
def get_embedding(text,model="text-embedding-3-small"):
    response=openai.embeddings.create(
        input=text,
        model=model
    )
    return response.data[0].embedding

### Retrieval function

In [14]:
qdrant_client=QdrantClient(url="http://localhost:6333")

def retrieve_data(query,k=5):
    query_embedding=get_embedding(query)
    results=qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k
    )
    retrieved_context_ids=[]
    retrieved_context=[]
    similarity_scores=[]
    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids":retrieved_context_ids,
        "retrieved_context":retrieved_context,
        "similarity_scores":similarity_scores
    }

retrieved_context=retrieve_data("What kind of earphones can I get?",k=10)

### Format retrieved context

In [19]:
def process_context(context):
    formatted_context=""
    for id,chunk in zip(context["retrieved_context_ids"],context["retrieved_context"]):
        formatted_context+=f"- {id}: {chunk}\n"
    
    return formatted_context

print(process_context(retrieved_context))
preprocessed_context=process_context(retrieved_context)

- B0CBMPG524: Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts 【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing.【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headphones providing pure balanced audio and clarity output for all music genres with soft audio and immerse yourself in the wonderful world of music.【Hear Your Surroundings】Open earbuds that rest on your ears without covering them, you can hear your music and your surroundings at the same time. Whether you are cycling, walking, runnin

### Create prompt function

In [18]:
def build_prompt(preprocessed_context,question):

    prompt= f"""
You are a shopping assistant that can answer questions about hte product in stock.
You will be given a question and a list of context.

Instructions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""
    return prompt

In [22]:
prompt=build_prompt(preprocessed_context,"What kind of earphones can I get?")
print(prompt)


You are a shopping assistant that can answer questions about hte product in stock.
You will be given a question and a list of context.

Instructions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
- B0CBMPG524: Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts 【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing.【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headphones providing pure balanced aud

### Generate Answer function

In [23]:
def generate_answer(prompt):

    response=openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role":"user","content":prompt}]
    )
    return response.choices[0].message.content

print(generate_answer(prompt))

You can get the following types of earphones:

1. **Open Ear Headphones** (B0CBMPG524) - Bluetooth 5.3 earbuds with an open-ear design, 60H playtime, and IPX7 waterproof rating, ideal for running and workouts.
  
2. **Wireless Earbuds** (B0B9FTVL58) - Bluetooth 5.3 headphones with microphone, 37H playback, deep bass, and IPX7 waterproof rating.

3. **Kids Headphones** (B0C142QS8X) - Over-ear, volume-limited headphones designed for children, compatible with various devices.

4. **Wireless Earbuds** (B0C6K1GQCF) - Bluetooth headphones with touch control, noise-canceling mic, 30H playtime, and also IPX7 waterproof.

5. **Bone Conduction Headphones** (B0BNHVLF7G) - Open-ear sports earphones using bone conduction technology, sweat-resistant, for workouts and outdoor activities.

6. **Game Earbuds** (B09WCFC5D9) - True wireless stereo headset designed for gaming with low latency, touch control, and deep bass.

7. **Kids Cat Earbuds** (B0B67ZFRPC) - Wired earphones designed for kids, featurin